## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
# Import the API key.
%load_ext dotenv
%dotenv
import os
gkey = os.getenv('G6_KEY')

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Cape Town,ZA,62.38,overcast clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
1,5,Rikitea,PF,71.33,few clouds,-23.1203,-134.9692,People ThankYou
2,13,Bethel,US,60.66,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
3,14,Vilhena,BR,66.60,overcast clouds,-12.7406,-60.1458,Hotel Mirage
4,20,Busselton,AU,60.39,few clouds,-33.6500,115.3333,Observatory Guest House


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(48.5, -120.0), zoom_level=1.8)
max_temp = vacation_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == 'Burien']
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'College Place']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Lumby']
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Powell River']
display(vacation_start, vacation_end, vacation_stop1, vacation_stop2, vacation_stop3)
type(vacation_start)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
71,245,Burien,US,67.93,clear sky,47.4704,-122.3468,Red Roof Inn Seattle Airport - SEATAC


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
71,245,Burien,US,67.93,clear sky,47.4704,-122.3468,Red Roof Inn Seattle Airport - SEATAC


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
141,531,College Place,US,70.56,clear sky,46.0493,-118.3883,"Holiday Inn Express Walla Walla, an IHG Hotel"


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
93,352,Lumby,CA,62.19,overcast clouds,50.2498,-118.969,Twin Creeks Motel


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,30,Powell River,CA,65.34,clear sky,49.8496,-124.5361,Rodmay Hotel


pandas.core.frame.DataFrame

In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = tuple(vacation_start.to_numpy()[0][5:7])
end = start
stop1 = tuple(vacation_stop1.to_numpy()[0][5:7])
stop2 = tuple(vacation_stop2.to_numpy()[0][5:7])
stop3 = tuple(vacation_stop3.to_numpy()[0][5:7])
print(start, end, stop1, stop2, stop3)
type(start)

(47.4704, -122.3468) (47.4704, -122.3468) (46.0493, -118.3883) (50.2498, -118.969) (49.8496, -124.5361)


tuple

In [12]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
waypoints = (stop1, stop2, stop3)
travelmode = ("DRIVING", "BICYCLING", "WALKING")
figure_layout = {'width': '700px', 'height': '700px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
start2end = gmaps.directions_layer(start, end, waypoints=waypoints, travel_mode=travelmode[0])
fig.add_layer(start2end)
fig

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', width='700px'))

In [13]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3], ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,245,Burien,US,67.93,clear sky,47.4704,-122.3468,Red Roof Inn Seattle Airport - SEATAC
1,531,College Place,US,70.56,clear sky,46.0493,-118.3883,"Holiday Inn Express Walla Walla, an IHG Hotel"
2,352,Lumby,CA,62.19,overcast clouds,50.2498,-118.9690,Twin Creeks Motel
3,30,Powell River,CA,65.34,clear sky,49.8496,-124.5361,Rodmay Hotel


In [14]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map.
figure_layout = {'width': '700px', 'height': '700px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', width='700px'))